<a href="https://colab.research.google.com/github/joeyfinley26/Project2/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2 - Ecommerce Data Analysis
### Joey Finley

---

In [ ]:
# authenticate with the google account used in console
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd
project_id = 'cohesive-pad-380023'

## Part 1: Products

In [ ]:
## This counts the total number of orders (not returned or cancelled) and the number of items for each.
sql = '''
SELECT 
  user_id,
  count(*) order_count 
FROM `bigquery-public-data.thelook_ecommerce.orders` 
where status not in ('Returned','Cancelled')
group by user_id
'''

order_count = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='user_id')

In [ ]:
order_count.value_counts()

order_count
1              46211
2              14459
3               4169
4               1535
dtype: int64

In [ ]:
## This displays each product and its name, and each time it was either added to a users cart or purchased
sql = '''
SELECT product_id, name, event_type
FROM
`bigquery-public-data.thelook_ecommerce.order_items` oi
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.events` e
ON 
oi.user_id = e.user_id
WHERE event_type = 'cart' OR event_type = 'purchase'
'''
product_web = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='product_id')

In [ ]:
product_web.head()

,name,event_type
product_id,,
13606,Elegant PASHMINA SCARF WRAP SHAWL STOLE,purchase
13606,Elegant PASHMINA SCARF WRAP SHAWL STOLE,cart
13606,Elegant PASHMINA SCARF WRAP SHAWL STOLE,cart
13606,Elegant PASHMINA SCARF WRAP SHAWL STOLE,cart
13606,Elegant PASHMINA SCARF WRAP SHAWL STOLE,cart


In [ ]:
## This summarizes the number of orders each product has
sql = '''
SELECT
  COUNT(*) order_id, 
  p.name
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
JOIN
  `bigquery-public-data.thelook_ecommerce.events` e
ON 
oi.user_id = e.user_id
GROUP BY 2 
ORDER BY order_id desc;
'''

product_orders = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='name')

In [ ]:
product_orders.head(20)

,order_id
name,
True Religion Men's Ricky Straight Jean,1316
Wrangler Men's Premium Performance Cowboy Cut Jean,1280
Wrangler Men's Genuine Tampa Cargo Short,1212
Puma Men's Socks,1043
Wrangler Men's Original Cowboy Cut Relaxed Fit Jean,965
Volcom Men's Nova Jean,946
Wrangler Men's Retro Straight Leg Jean,927
7 For All Mankind Men's Standard Classic Straight Leg Jean,910
7 For All Mankind Men's Austyn Relaxed Straight Leg Jean,894


In [ ]:
## This shows 20% of products to cut based on user's web habits. These products had the highest difference between the amount of times they were added in a cart
  ## and the amount of times they were purchased. There are 29,047 products. 20% of that is 5,809. 

sql = '''
SELECT product_id, name,
  SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as num_cart,
  SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as num_purchase,
  (SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END)) - (SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END)) as difference
FROM
`bigquery-public-data.thelook_ecommerce.order_items` oi
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
INNER JOIN
  `bigquery-public-data.thelook_ecommerce.events` e
ON 
oi.user_id = e.user_id
WHERE event_type = 'cart' OR event_type = 'purchase'
GROUP BY 1,2
ORDER BY difference DESC
LIMIT 5809
;
'''
bottom_products = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='product_id')

In [ ]:
## These are the products to remove
bottom_products.head(5809)

,name,num_cart,num_purchase,difference
product_id,,,,
23981,Carhartt Active Jacket,233,81,152
16582,Key Industries Men's Big-Tall Long Sleeve Heav...,213,73,140
24299,ADIDAS ADI FIREBIRD TRACK JACKET TOP MENS X46181,213,75,138
14263,Dearfoams Women's Handknit Infinity Scarf,194,68,126
24620,Wigwam Men's Cool-Lite Pro Crew Socks,187,65,122
...,...,...,...,...
20457,Allegra K Mens NEW Stylish Adjustable Buckle B...,67,27,40
23619,KR3W Beckley Shorts - Grey,62,22,40
14842,A Pea in the Pod: Spanx Power Mama Panty,65,25,40


# Part 2: Users

In [ ]:
## This displays each users' web habits, and summarizes the number of items they purchased, the number of items added to their carts, and the difference between the two
## (the difference between how many items were added to the cart and how many of them were actually purchased.)
sql = '''
SELECT
  user_id,
  SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) as num_cart,
  SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) as num_purchase,
  (SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END)) - (SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END)) as difference
FROM
  `bigquery-public-data.thelook_ecommerce.events`
WHERE
  user_id IS NOT NULL
GROUP BY
  1
'''

user_web = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='user_id')

In [ ]:
user_web.head()

,num_cart,num_purchase,difference
user_id,,,
10493,22,8,14
52353,30,10,20
47650,14,6,8
4754,22,8,14
45175,19,7,12


In [ ]:
## Each user and the count of orders they purchased (all statuses, including shipped, completed, processing, returned)
sql = '''
SELECT
  user_id,
  COUNT(*) order_id
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
GROUP BY 1;
'''

user_orders = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='order_id')

In [ ]:
user_orders.head()

,user_id
order_id,
10,2114
9,52309
9,78270
9,14184
9,35777


In [ ]:
## Web Behavior: this categorizes each user based on the traffic source they used and the number of purchases based off the corresponding traffic source

sql = '''
SELECT
  user_id,
  traffic_source,
  COUNT(*) purchase_count
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.users` u
ON
  oi.product_id = u.id
WHERE
  status NOT IN ('Returned','Cancelled')
GROUP BY 1,2;
'''

user_source = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='user_id')

In [ ]:
user_source.head()

,traffic_source,purchase_count
user_id,,
4065,Email,1
26080,Email,1
48851,Email,1
86204,Email,1
27276,Email,1


In [ ]:
## Returns Behavior: this categorizes each user based on the traffic source they used and the number of times they made a returns based off the corresponding traffic source

sql = '''
SELECT
  user_id,
  traffic_source,
  COUNT(*) purchase_count
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.users` u
ON
  oi.product_id = u.id
WHERE
  status IN ('Returned')
GROUP BY 1,2;
'''

return_source = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='user_id')


In [ ]:
return_source.head()

,traffic_source,purchase_count
user_id,,
64911,Email,1
16813,Facebook,1
20027,Search,3
25356,Organic,1
67214,Search,4


In [17]:
## Traffic Sources: this shows the number of purchases generated from each traffic source. 
sql = '''
SELECT
  traffic_source,
  COUNT(*) purchase_count
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.users` u
ON
  oi.product_id = u.id
WHERE
  status NOT IN ('Returned', 'Cancelled')
GROUP BY 1
ORDER BY traffic_source DESC;
'''

purchase_source = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='traffic_source')

In [20]:
purchase_source.head()

,purchase_count
traffic_source,
Search,95445
Organic,20466
Facebook,7941
Email,6883
Display,5390


#### Categorizing each user based off traffic source allows businesses to not only view the amount of times a user returned or purchased an item based on the traffic source, but also allows them to view what areas they should focus on for a marketing standpoint, where the most traffic is flowing from. As shown above, the traffic source that yields the most purchases is general searches, followed by organic. 

In [ ]:
## (BOTH PARTS) This shows the specific product names that each user purchased, and the count of each product 
  ## (does not include returned or cancelled orders)

sql = '''
SELECT
  user_id,
  p.name,
  COUNT(*) purchase_count
FROM
  `bigquery-public-data.thelook_ecommerce.order_items` oi
JOIN
  `bigquery-public-data.thelook_ecommerce.products` p
ON
  oi.product_id = p.id
WHERE
  status NOT IN ('Returned','Cancelled')
GROUP BY 1,2
ORDER BY purchase_count desc;
'''

user_products = pd.io.gbq.read_gbq(sql,project_id=project_id,
                                      index_col='user_id')

In [ ]:
user_products.head()

,name,purchase_count
user_id,,
52319,Papi Men's Fiesta Cool Dry Brazilian Trunk Brief,2
47926,Hello Kitty Juniors Dot Print Short Set,2
89236,Frederick's of Hollywood Stretch Lace High Wai...,2
96435,N2N Bodywear BT9 - Soft and Luxurious Brushed ...,2
43064,Matix Men's Gripper Denim Pant,2
